# Transcribe Recordings
paid service [Goodtape](http://goodtape.io) will be used for the transcriptions

## Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [12]:
import requests
import pandas as pd
import os
import shutil
import numpy as np
import io
import json
import math

from requests.utils import quote

# audio
from pydub import AudioSegment

# local imports
from lib_henryk.logger import *
from lib_henryk.config import *

from lib_henryk.transcription import transcription

## Params

In [27]:
# load api keys
from dotenv import load_dotenv
_ = load_dotenv()

# retrieve new token by opening 'https://webhook.site/' in incognito mode
WEBHOOKS_TOKEN_ID = 'f7477993-66da-4747-b9c8-6bc5c302ef3c'
GOODTAPE_API_KEY = os.getenv("GOODTAPE_API_KEY")

## Load Dataset

In [4]:
df = pd.read_csv(FILE_AUDIO_STATS_CSV)
df_transcription = df.copy()
df_transcription['transcription_id'] = ''

## Run Transcriptions

In [28]:
# load sync dataframe
if os.path.exists(FILE_TRANSCRIPTION_CSV):
    print(f'transcription log exists, reading it from {FILE_TRANSCRIPTION_CSV}')
    df_transcription = pd.read_csv(FILE_TRANSCRIPTION_CSV)

transcription log exists, reading it from ../../data/processed/henryk_audio_transcription.csv


In [29]:
# perform transcriptions
transcription.submit_transcriptions_goodtape(
    df=df_transcription, 
    webhooks_token_id=WEBHOOKS_TOKEN_ID,
    goodtape_api_key=GOODTAPE_API_KEY,
    path_recordings=DIR_WIADOMOSCI_DO_HENRYCZKA,
    start=0, stop=500,
    verbose=False
)

processing [467]: Henryk 2024-03-20 Tatuś opowiada po co jest gorączka a Miś Uszatek spotyka krasnoludki.m4a
processing [469]: Henryk 2024-03-22 Tatuś opowiada o domku na kółkach i o wycieczkach a Miś Uszatek widzi raka.m4a
processing [470]: Henryk 2024-03-24 Tatuś opowiada o odpoczynku w górach a Miś Uszatek żegluje.m4a
processing [471]: Henryk 2024-03-25 Tatuś opowiada o tym jak odpoczywać a zabawki w przedszkolu śpiewają piosenkę...
processing [472]: Henryk 2024-03-27 Tatuś opowoada o lazanii a Miś Uszatek ma gościa z Francji.m4a
processing [473]: Henryk 2024-03-29 Tatuś opowiada o kolorowaniu jajek a Miś Uszatek odwiedza lale.m4a
processing [474]: Henryk 2024-03-30 Tatuś opowiada o święceniu koszyczka wielkanocnego a Miś Uszatek gasi pożar.m...
processing [475]: Henryk 2024-03-31 Tatuś opowiada o śniadaniu Wielkanocnym a Miś Uszatek spotyka kozy.m4a
processing [477]: Henryk 2024-04-01 Tatuś opowiada o lanym poniedziałku a Miś Uszatek wyrusza z cyrkiem 

## Fetch Transcriptions JSONs

In [7]:
# need to wait until transcriptions are available
exit_cell()

StopExecution: stopped

In [43]:
# retrieve goodtape server responses from webhooks
transcription.retrieve_responses_goodtape_via_webhooks(
    df=df_transcription,
    webhooks_token_id=WEBHOOKS_TOKEN_ID, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON
)

# process json to text
transcription.process_transcriptions_json_to_text(
    df=df_transcription, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT,
    verbose=False
)

found 5 requests to fetch
retrieved transcription [0]: f42af9a2-d89c-422b-8a7f-b0a5f47f47c7 -> file [496]: Henryk 2024-05-01 Tatuś opowiada o dniu pracy a Trąbiący dino...
retrieved transcription [1]: 8f49eaac-4d3b-46a5-992e-e6acbe20cb45 -> file [495]: Henryk 2024-04-29 Tata opowiada o konkursie i nagrodach a Gastor...
retrieved transcription [2]: 8dad1ecd-1d8e-4b7e-a892-9c5a96650c0c -> file [497]: Henryk 2024-05-03 Tatuś opowiada o Rycerzach i Zamkach a Prapłe...
retrieved transcription [3]: 5aa57e03-6c6d-4d41-8636-1fcf0043bf62 -> file [494]: Henryk 2024-04-28 Tatuś opowiada o malowaniu krzesełek a Diplod...
retrieved transcription [4]: b99141db-d62c-4aca-bf1c-46f594645cf8 -> file [491]: Henryk 2024-04-22 Tatuś opowiada o Dniu Ziemi a Nosorożec Nono...
5 transcriptions were retrieved
5 new transcriptions were processed, there are 498 transcriptions available


## Cleanup missing requests
- transcription ids for the requests that were lost will be removed, only those that were processed will remain

In [44]:
# cleanup stale transcription ids - but work on a copy of the log
df_transcriptions_to_write = transcription.get_cleaned_up_transcriptions(
    df=df_transcription, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT
)

# save transcription log
df_transcriptions_to_write.to_csv(FILE_TRANSCRIPTION_CSV, index=False)

there are 498 processed and valid transcriptions
*** all 498 recordings were transcribed ***
